In [ ]:
!pip install -U ultralytics
!pip install -U ipywidgets

Splitting the dataset:
train ratio: 50%
val ratio: 25%
test ratio: 25%

In [ ]:
import os
import shutil

# Caminho para o dataset já organizado
dataset_path = '/kaggle/input/chromosome-oriented-bb/dataset'

# Diretório de saída (onde você quer copiar os dados)
output_path = '/kaggle/working'

# Conjuntos
splits = ['train', 'val', 'test']

for split in splits:
    src_img_dir = os.path.join(dataset_path, 'images', split)
    src_lbl_dir = os.path.join(dataset_path, 'labels', split)

    dst_img_dir = os.path.join(output_path, split, 'images')
    dst_lbl_dir = os.path.join(output_path, split, 'labels')

    os.makedirs(dst_img_dir, exist_ok=True)
    os.makedirs(dst_lbl_dir, exist_ok=True)

    for fname in os.listdir(src_img_dir):
        if not fname.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue

        # Copia imagem
        shutil.copy(os.path.join(src_img_dir, fname),
                    os.path.join(dst_img_dir, fname))

        # Copia label correspondente (se existir)
        label_name = os.path.splitext(fname)[0] + '.txt'
        label_src = os.path.join(src_lbl_dir, label_name)

        if os.path.exists(label_src):
            shutil.copy(label_src, os.path.join(dst_lbl_dir, label_name))
        else:
            print(f'⚠️ Sem label para {fname}')

print("Cópia concluída para /kaggle/working/{train,val,test}")

In [ ]:
import os

working_path = '/kaggle/working'

def count_images(split):
    images_dir = os.path.join(working_path, split, 'images')
    return len([f for f in os.listdir(images_dir) if f.endswith(('.png', '.jpg', '.jpeg'))])

train_count = count_images('train')
val_count = count_images('val')
test_count = count_images('test')
total_count = train_count + val_count + test_count

print(f"Total de imagens em /kaggle/working: {total_count}")
print(f"Imagens de treino: {train_count}")
print(f"Imagens de validação: {val_count}")
print(f"Imagens de teste: {test_count}")


In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np

dataset_path = '/kaggle/input/chromosome-oriented-bb/dataset'
split = 'test'
image_id = '7.png'

if not image_id.endswith('.png'):
    image_id += '.png'

image_path = os.path.join(dataset_path, 'images', split, image_id)
label_path = os.path.join(dataset_path, 'labels', split, image_id.replace('.png', '.txt'))

image = cv2.imread(image_path)
if image is None:
    raise FileNotFoundError(f'Imagem não encontrada: {image_path}')

image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
h, w, _ = image.shape

if os.path.exists(label_path):
    with open(label_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 9:
                continue

            cls_id = int(parts[0])
            coords = list(map(float, parts[1:]))

            # Convertendo para pixels
            points = []
            for i in range(0, 8, 2):
                x = int(coords[i] * w)
                y = int(coords[i + 1] * h)
                points.append([x, y])

            points = np.array(points)

            # Desenhar polígono
            cv2.polylines(image, [points], isClosed=True, color=(255, 0, 0), thickness=2)

            # Texto da classe no primeiro ponto
            cv2.putText(image, f"Class {cls_id}", (points[0][0], points[0][1] - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
else:
    print(f'Nenhum label encontrado: {label_path}')

plt.figure(figsize=(8, 8))
plt.imshow(image)
plt.axis('off')
plt.title(f"{split.upper()} - {image_id}")
plt.show()


In [ ]:
import torch
from ultralytics import YOLO
from torch import nn

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = YOLO('yolo11m-obb.pt')

device_ids = [0, 1] 

config_path = '/kaggle/input/config/config.yaml'

results = model.train(
    data=config_path,
    epochs=200,
    task='obb',
    batch=4,
    imgsz=1080,
    device=0
)

In [ ]:
metrics = model.val(data=config_path, task='obb', split="test")
print(metrics.results_dict)

In [ ]:
import os
from ultralytics import YOLO

input_folder = "/kaggle/working/test/images"
output_folder = "/kaggle/working/predictions"

os.makedirs(output_folder, exist_ok=True)

supported_formats = (".jpg", ".jpeg", ".png", ".bmp")  # Formatos suportados
image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(supported_formats)]

for image_file in image_files:
    input_path = os.path.join(input_folder, image_file)

    model.predict(
        source=input_path, 
        save_txt=True,
        task='obb',
        conf=0.5,
        project=output_folder,
        iou=0.5,
        name=os.path.splitext(image_file)[0],
        show_labels=False,
        device=0
    )

In [ ]:
import os
import cv2
import numpy as np

predictions_folder = "/kaggle/working/predictions"
output_folder = "/kaggle/working/predictions_with_boxes"
input_images_folder = "/kaggle/working/test/images"

os.makedirs(output_folder, exist_ok=True)

for subdir in os.listdir(predictions_folder):
    subdir_path = os.path.join(predictions_folder, subdir)
    if not os.path.isdir(subdir_path):
        continue

    label_path = os.path.join(subdir_path, "labels", f"{subdir}.txt")
    image_path = os.path.join(input_images_folder, f"{subdir}.png")

    if not os.path.exists(label_path):
        print(f"Arquivo de labels ausente para: {subdir}")
        continue
    if not os.path.exists(image_path):
        print(f"Imagem original ausente para: {subdir}")
        continue

    image = cv2.imread(image_path)
    height, width, _ = image.shape

    with open(label_path, "r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 9:
                print(f"Formato inválido na label {label_path}: {line}")
                continue

            cls = int(parts[0])
            coords = list(map(float, parts[1:]))

            # Converter coordenadas normalizadas para pixels
            points = []
            for i in range(0, 8, 2):
                x = int(coords[i] * width)
                y = int(coords[i + 1] * height)
                points.append([x, y])

            points = np.array(points)

            # Desenhar o polígono (caixa orientada)
            cv2.polylines(image, [points], isClosed=True, color=(0, 255, 0), thickness=2)

            # Opcional: escrever a classe no primeiro ponto
            cv2.putText(image, f"Class {cls}", (points[0][0], points[0][1] - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    # Salvar a imagem anotada
    output_image_path = os.path.join(output_folder, f"{subdir}.jpg")
    cv2.imwrite(output_image_path, image)
    print(f"Imagem anotada salva em: {output_image_path}")

print("Processamento concluído!")


In [ ]:
import os
import cv2
import numpy as np

def calculate_obb_iou(box1, box2):
    poly1 = np.array(box1, dtype=np.float32).reshape((4, 2))
    poly2 = np.array(box2, dtype=np.float32).reshape((4, 2))

    inter_area, _ = cv2.intersectConvexConvex(poly1, poly2)
    if inter_area == 0:
        return 0.0

    area1 = cv2.contourArea(poly1)
    area2 = cv2.contourArea(poly2)
    union_area = area1 + area2 - inter_area

    return inter_area / union_area if union_area > 0 else 0

ground_truth_folder = "/kaggle/working/test/labels"
predictions_folder = "/kaggle/working/predictions"
input_images_folder = "/kaggle/working/test/images"
output_folder = "/kaggle/working/iou_output"

os.makedirs(output_folder, exist_ok=True)

iou_threshold = 0.5
true_positives = 0
false_positives = 0
false_negatives = 0

for gt_file in os.listdir(ground_truth_folder):
    gt_path = os.path.join(ground_truth_folder, gt_file)
    pred_subfolder = os.path.join(predictions_folder, os.path.splitext(gt_file)[0])
    pred_path = os.path.join(pred_subfolder, "labels", gt_file)

    if not os.path.exists(pred_path):
        print(f"Predição ausente para: {gt_file}")
        continue

    image_name = os.path.splitext(gt_file)[0]
    image_path = os.path.join(input_images_folder, image_name + ".png")

    if not os.path.exists(image_path):
        print(f"Imagem ausente: {image_path}")
        continue

    image = cv2.imread(image_path)
    if image is None:
        print(f"Erro ao carregar imagem: {image_path}")
        continue

    height, width, _ = image.shape

    def parse_box(line):
        coords = list(map(float, line.strip().split()[1:]))
        # Normalizar para coordenadas absolutas
        points = [(coords[i] * width, coords[i + 1] * height) for i in range(0, 8, 2)]
        return points

    with open(gt_path, "r") as f:
        gt_boxes = [parse_box(line) for line in f]

    with open(pred_path, "r") as f:
        pred_boxes = [parse_box(line) for line in f]

    matched = set()
    for gt_box in gt_boxes:
        gt_polygon = np.array(gt_box, dtype=np.int32)
        detected = False
        for i, pred_box in enumerate(pred_boxes):
            if i in matched:
                continue
            iou = calculate_obb_iou(gt_box, pred_box)
            if iou >= iou_threshold:
                matched.add(i)
                true_positives += 1
                cv2.polylines(image, [np.array(pred_box, dtype=np.int32)], isClosed=True, color=(0, 255, 0), thickness=1)
                detected = True
                break
        if not detected:
            false_negatives += 1
            cv2.polylines(image, [gt_polygon], isClosed=True, color=(255, 0, 0), thickness=1)

    for i, pred_box in enumerate(pred_boxes):
        if i not in matched:
            false_positives += 1
            cv2.polylines(image, [np.array(pred_box, dtype=np.int32)], isClosed=True, color=(0, 0, 255), thickness=1)

    output_image_path = os.path.join(output_folder, image_name + "_output.jpg")
    cv2.imwrite(output_image_path, image)

print(f"True Positives: {true_positives}")
print(f"False Positives: {false_positives}")
print(f"False Negatives: {false_negatives}")


In [ ]:
import os
import shutil

# Pastas que você quer compactar
folders_to_zip = ['/kaggle/working/test', '/kaggle/working/iou_output', '/kaggle/working/predictions']

# Caminho para a pasta temporária que vai agrupar todas
temp_dir = '/kaggle/working/zip_temp'

# Caminho final do arquivo zip
zip_path = '/kaggle/working/output.zip'

# Garantir que a pasta temporária esteja limpa
if os.path.exists(temp_dir):
    shutil.rmtree(temp_dir)
os.makedirs(temp_dir)

# Copiar as pastas desejadas para a pasta temporária
for folder in folders_to_zip:
    folder_name = os.path.basename(folder)
    shutil.copytree(folder, os.path.join(temp_dir, folder_name))

# Compactar a pasta temporária
shutil.make_archive(zip_path.replace('.zip', ''), 'zip', temp_dir)

# (Opcional) Remover a pasta temporária
shutil.rmtree(temp_dir)

In [ ]:
import os
import numpy as np
from scipy.optimize import linear_sum_assignment

# Caminhos
pred_dir = "/kaggle/working/predictions"
gt_dir = "/kaggle/working/test/labels"

# Função para calcular IoU
def compute_iou(box1, box2):
    x1_min, y1_min, x1_max, y1_max = box1
    x2_min, y2_min, x2_max, y2_max = box2

    xi1 = max(x1_min, x2_min)
    yi1 = max(y1_min, y2_min)
    xi2 = min(x1_max, x2_max)
    yi2 = min(y1_max, y2_max)

    inter_area = max((xi2 - xi1), 0) * max((yi2 - yi1), 0)
    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)
    union_area = box1_area + box2_area - inter_area

    if union_area == 0:
        return 0.0
    return inter_area / union_area

# Função para ler boxes no seu formato
def read_boxes(file_path, img_w=640, img_h=640):
    with open(file_path, "r") as f:
        lines = f.readlines()
        if not lines:
            return []
        items = list(map(float, lines[0].strip().split()))
        boxes = []
        for i in range(1, len(items), 4):
            if i + 3 < len(items):
                x_c, y_c, w, h = items[i:i + 4]
                x_c *= img_w
                y_c *= img_h
                w *= img_w
                h *= img_h
                x_min = x_c - w / 2
                y_min = y_c - h / 2
                x_max = x_c + w / 2
                y_max = y_c + h / 2
                boxes.append([x_min, y_min, x_max, y_max])
        return boxes

# Verificação por imagem
perfect_matches = 0
total_images = 0

for img_folder in os.listdir(pred_dir):
    pred_file = os.path.join(pred_dir, img_folder, "labels", f"{img_folder}.txt")
    gt_file = os.path.join(gt_dir, f"{img_folder}.txt")
    
    if not os.path.exists(pred_file) or not os.path.exists(gt_file):
        continue

    preds = read_boxes(pred_file)
    gts = read_boxes(gt_file)

    total_images += 1

    if len(gts) == 0:
        # Se não há GT, considera como perfeitamente correto se também não houver predição
        if len(preds) == 0:
            perfect_matches += 1
        continue

    if len(preds) == 0:
        # GT não vazio mas predição vazia → falhou
        continue

    # Matriz de IoUs (GT x Pred)
    iou_matrix = np.zeros((len(gts), len(preds)))

    for i, gt in enumerate(gts):
        for j, pred in enumerate(preds):
            iou_matrix[i, j] = compute_iou(gt, pred)

    # Matching ótimo (Hungarian)
    cost_matrix = 1 - iou_matrix  # Menor custo = maior IoU
    gt_idxs, pred_idxs = linear_sum_assignment(cost_matrix)

    matches = 0
    for gt_idx, pred_idx in zip(gt_idxs, pred_idxs):
        if iou_matrix[gt_idx, pred_idx] >= 0.5:
            matches += 1

    # Checa se todos os GTs foram detectados (não importa se houve falsas predições além disso)
    if matches == len(gts):
        perfect_matches += 1

print(f"Imagens com 100% de acerto (todas GT detectadas): {perfect_matches}/{total_images}")
